In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import torch

In [331]:
class Wav2vec(nn.Module):

    def __init__(self):
        super(Wav2vec, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        activation = nn.ReLU()
        dropout = 0.0
        self.encoder = Encoder(activation, dropout)
        self.context = Context(10,10,3,0.5,nn.ReLU())
        
        # Calculate offset for prediction module
        def calc_offset(): 
            jin = 0
            rin = 0
            for layer in next(self.encoder.children()):
                if layer.__class__.__name__ == 'Conv1d': 
                    k = layer.kernel_size[0]
                    stride = layer.stride[0]
                    if rin == 0:
                        rin = k
                    rin = rin + (k - 1) * jin
                    if jin == 0:
                        jin = stride
                    else:
                        jin *= stride
            offset = math.ceil(rin / jin)

            return int(offset)
        
        self.offset = calc_offset()
        

    def forward(self, x):
        z = self.encoder(x)
        c = self.context(z)
        # x = x.view(-1, self.num_flat_features(x))
        return z, c
    
class Encoder(nn.Module):
    def __init__(self, activation, dropout):
        super(Encoder, self).__init__()
        self.in_c = 10
        
        # Hardcoded architecture, as the blocks are different
        self.encoder = nn.Sequential(nn.Conv1d(in_channels=1, out_channels=self.in_c, kernel_size=10, stride=5),
                                     nn.Dropout(p=dropout),
                                     nn.GroupNorm(1, self.in_c),  # Affine, what to do?
                                     activation,
                                     # 2nd layer
                                     nn.Conv1d(in_channels=self.in_c, out_channels=self.in_c, kernel_size=8, stride=4),
                                     nn.Dropout(p=dropout),
                                     ## See norm_block - FB_repo
                                     nn.GroupNorm(1, self.in_c),  # Affine, what to do?
                                     activation,
                                     # 3rd layer
                                     nn.Conv1d(in_channels=self.in_c, out_channels=self.in_c, kernel_size=4, stride=2),
                                     nn.Dropout(p=dropout),
                                     nn.GroupNorm(1, self.in_c),  # Affine, what to do?
                                     activation,
                                     # Fourth layer
                                     nn.Conv1d(in_channels=self.in_c, out_channels=self.in_c, kernel_size=4, stride=2),
                                     nn.Dropout(p=dropout),
                                     nn.GroupNorm(1, self.in_c),  # Affine, what to do?
                                     activation,
                                     # Fifth layer
                                     nn.Conv1d(in_channels=self.in_c, out_channels=self.in_c, kernel_size=4, stride=2),
                                     nn.Dropout(p=dropout),
                                     nn.GroupNorm(1, self.in_c),  # Affine, what to do?
                                     activation)
    def log_compression(self, x):
        # https://www.edn.com/log-1-x-compression/
        x = x.abs()
        x = x + 1
        return x.log()

    def forward(self, x):
        x = self.encoder(x)
        x = self.log_compression(x)
        # TODO implement skipped connections?
        return x
    
class Context(nn.Module):
    def __init__(self, n_in, n_out, k,dropout, activation, layers=10):
        super(Context, self).__init__()

        # All block are the same, so create using a function
        def conv_block(n_in, n_out, k, dropout, activation):
            return nn.Sequential(
                nn.Conv1d(n_in, n_out, k, padding=1),
                nn.Dropout(p=dropout),
                nn.GroupNorm(1, n_out),
                activation
            )

        # Holder for conv layers
        self.conv = nn.ModuleList()
        
        # Create #layers number of conv-blocks
        for i in range(0, layers):
            self.conv.append(conv_block(n_in, n_out, k, dropout, activation))
        self.conv = nn.Sequential(*self.conv)

    def forward(self, x):
        x = self.conv(x)
        return x


In [332]:
m = Wav2vec()

In [333]:
waveform, sample_rate = torchaudio.load("wav_16k_example.wav")

In [334]:
torch.unsqueeze(waveform, 1).shape

torch.Size([1, 1, 31440])

In [335]:
(31440/194)

162.06185567010309

In [336]:
(162*31/1000)

5.022

In [337]:
m = Wav2vec()
z,c = m(waveform.unsqueeze(1))

In [338]:
# z 
z.shape

torch.Size([1, 10, 194])

In [339]:
c.shape

torch.Size([1, 10, 194])

# Prediction class

In [340]:
class Prediction(nn.Module):
    def __init__(self, predictions_steps):
        in_dim = 10 # antal channels fra aggregator UPDATE TO USE AS ARGUMENT IN CONSTRUCTOR
        out_dim = 10 # antal outchannels fra encoder UPDATE TO USE AS ARGUMENT IN CONSTRUCTOR
        prediction_steps = 12 # Not sure what this is? It's an argument UPDATE TO USE AS ARGUMENT IN CONSTRUCTOR
        super(Prediction, self).__init__()
        self.transpose_context = nn.ConvTranspose2d(in_dim, out_dim, (1, prediction_steps))
        self.sample_distance = None
        self.n_negatives = 1
        
        
    def sample_negatives(self, y): 
        bsz, fsz, tsz = y.shape

        y = y.transpose(0, 1)  # BCT -> CBT
        y = y.contiguous().view(fsz, -1)  # CBT => C(BxT)

        cross_high = tsz * bsz
        high = tsz if self.sample_distance is None else min(tsz, self.sample_distance)
        assert high > 1

        neg_idxs = torch.randint(low=0, high=high, size=(bsz, self.n_negatives * tsz))

        with torch.no_grad():
            if self.n_negatives > 0:
                tszs = (
                    # REMEMBER TO INCLUDE BUFFERED_ARANGE FROM UTIL
                    buffered_arange(tsz)
                    .unsqueeze(-1)
                    .expand(-1, self.n_negatives)
                    .flatten()
                )

                neg_idxs = torch.randint(
                    low=0, high=high - 1, size=(bsz, self.n_negatives * tsz)
                )
                neg_idxs[neg_idxs >= tszs] += 1


        for i in range(1, bsz):
            neg_idxs[i] += i * high

        negs = y[..., neg_idxs.view(-1)]
        negs = negs.view(
            fsz, bsz, self.n_negatives + 0, tsz
        ).permute(
            2, 1, 0, 3
        )  # to NxBxCxT
        return negs

    
    def forward(self, c, z):
        c = c.unsqueeze(-1)
        # Transpose to give steps predictions into the future
        c = self.transpose_context(c)
        
        
        

In [341]:
def buffered_arange(max):
    if not hasattr(buffered_arange, "buf"):
        buffered_arange.buf = torch.LongTensor()
    if max > buffered_arange.buf.numel():
        buffered_arange.buf.resize_(max)
        torch.arange(max, out=buffered_arange.buf)
    return buffered_arange.buf[:max]

In [342]:
in_dim = 10 # antal channels fra aggregator 
out_dim = 10 # antal outchannels fra encoder
prediction_steps = 12 # Not sure what this is? It's an argument 
hk = nn.ConvTranspose2d(in_dim, out_dim, (1, prediction_steps))

In [343]:
c.shape

torch.Size([1, 10, 194])

In [344]:
hk(c.unsqueeze(-1)).shape

torch.Size([1, 10, 194, 12])

In [345]:
torch.normal(0.5, 0.5, size=(1, 4,))

tensor([[-0.1214,  1.1560,  0.8778, -0.2357]])

In [346]:
a = torch.tensor([1,2])
b = torch.tensor([3,4])

torch.cat([a,b], dim=0)

tensor([1, 2, 3, 4])

In [347]:
c.shape

torch.Size([1, 10, 194])

In [348]:
c.unsqueeze(0).shape

torch.Size([1, 1, 10, 194])

# Sample negatives

In [349]:
x = c.clone()
y = z.clone()

In [350]:
sample_distance = None
n_negatives = 1
def sample_negatives(y): 
    bsz, fsz, tsz = y.shape

    y = y.transpose(0, 1)  # BCT -> CBT
    y = y.contiguous().view(fsz, -1)  # CBT => C(BxT)

    cross_high = tsz * bsz
    high = tsz if sample_distance is None else min(tsz, sample_distance)
    assert high > 1

    neg_idxs = torch.randint(low=0, high=high, size=(bsz, n_negatives * tsz))

    with torch.no_grad():
        if n_negatives > 0:
            tszs = (
                # REMEMBER TO INCLUDE BUFFERED_ARANGE FROM UTIL
                buffered_arange(tsz)
                .unsqueeze(-1)
                .expand(-1, n_negatives)
                .flatten()
            )

            neg_idxs = torch.randint(
                low=0, high=high - 1, size=(bsz, n_negatives * tsz)
            )
            neg_idxs[neg_idxs >= tszs] += 1


    for i in range(1, bsz):
        neg_idxs[i] += i * high

    negs = y[..., neg_idxs.view(-1)]
    negs = negs.view(
        fsz, bsz, n_negatives + 0, tsz
    ).permute(
        2, 1, 0, 3
    )  # to NxBxCxT
    return negs

In [351]:
x.shape

torch.Size([1, 10, 194])

In [352]:
# TEST FUNCTION FOR PROJECT_STEPS
in_dim = 10 # antal channels fra aggregator 
out_dim = 10 # antal outchannels fra encoder
prediction_steps = 12 # Not sure what this is? It's an argument 
hk = nn.ConvTranspose2d(in_dim, out_dim, (1, prediction_steps))

# Unsqueeze before transposing (creating prediction_steps)
x = x.unsqueeze(-1)
x = hk(x)  # self.project_steps

negatives = sample_negatives(y)
y = y.unsqueeze(0)

In [353]:
y.shape

torch.Size([1, 1, 10, 194])

In [354]:
negatives.shape

torch.Size([1, 1, 10, 194])

In [355]:
targets = torch.cat([y, negatives], dim=0)
targets.shape

torch.Size([2, 1, 10, 194])

In [356]:
x.shape

torch.Size([1, 10, 194, 12])

In [357]:
targets = torch.cat([y, negatives], dim=0)  # Copies x B x C x T
copies = targets.size(0)
copies

2

In [358]:
bsz, dim, tsz, steps = x.shape
offset = m.offset
steps = min(steps, tsz - offset)
steps

12

In [369]:
steps = min(steps, tsz - offset)
infonce = False
balanced_classes = False

predictions = x.new(
    bsz * copies * (tsz - offset + 1) * steps
    - ((steps + 1) * steps // 2) * copies * bsz
)


In [371]:
predictions.shape

torch.Size([42])

In [ ]:
if infonce:
    labels = predictions.new_full(
        (predictions.shape[0] // copies,), 0, dtype=torch.long
    )
else:
    labels = torch.zeros_like(predictions)
weights = (
    torch.full_like(labels, 1 / n_negatives)
    if balanced_classes and not infonce
    else None
)

In [368]:
start = end = 0
for i in range(steps):
    offset = i + offset
    end = start + (tsz - offset) * bsz * copies
    print(end)
    

118
116
112
106
98
88
76
62
46
28
8
-14


In [ ]:
if infonce:
        predictions[start:end] = torch.einsum(
            "bct,nbct->tbn", x[..., :-offset, i], targets[..., offset:]
        ).flatten()
    else:
        pos_num = (end - start) // copies
        predictions[start:end] = torch.einsum(
            "bct,nbct->nbt", x[..., :-offset, i], targets[..., offset:]
        ).flatten()
        labels[start : start + pos_num] = 1.0
        if weights is not None:
            weights[start : start + pos_num] = 1.0
    start = end
#assert end == predictions.numel(), "{} != {}".format(end, predictions.numel())

if infonce:
    predictions = predictions.view(-1, copies)
else:
    if weights is not None:
        labels = (labels, weights)

In [365]:
predictions.shape

torch.Size([4452])

In [367]:
labels.shape

torch.Size([4452])